In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
print ("testing")

testing


In [4]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
train = datasets.ImageFolder(root='10B-3D-Uni/train', transform=transform)
val = datasets.ImageFolder(root='10B-3D-Uni/val', transform=transform)
test = datasets.ImageFolder(root='10B-3D-Uni/test', transform=transform)

train_loader = DataLoader(train, batch_size=16, shuffle=True)
val_loader = DataLoader(val, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

print(f"Training set size: {len(train)} images")
print(f"Validation set size: {len(val)} images")
print(f"Testing set size: {len(test)} images")

Training set size: 1200 images
Validation set size: 400 images
Testing set size: 400 images


In [14]:
from torchvision.models import resnet18, ResNet18_Weights

resnet18 = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

resnet18.fc = nn.Linear(resnet18.fc.in_features, 2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
resnet18 = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(resnet18.parameters(), lr=0.001)
optimizer = optim.AdamW(resnet18.parameters(), lr=0.001, weight_decay=0.01)

Using device: cuda


In [6]:
def evaluate_model(model, data_loader):
    """Evaluate model on the provided data loader"""
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total * 100
    avg_loss = running_loss / len(data_loader)
    return accuracy, avg_loss

In [12]:
import torch.optim.lr_scheduler as lr_scheduler

num_epochs = 3
best_val_loss = float('inf')
patience = 8 
patience_counter = 0
temp = None
print("Starting training...")

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

for epoch in range(num_epochs):

    resnet18.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_accuracy = correct / total * 100 
    train_loss = running_loss / len(train_loader)
    

    val_accuracy, val_loss = evaluate_model(resnet18, val_loader) #check overfitting
    

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0 
        temp = resnet18.state_dict()
    else:
        patience_counter += 1 
        print(f"patience counter {patience_counter}")

    scheduler.step(val_loss)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}]')
    print(f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
    print(f'Current LR: {optimizer.param_groups[0]["lr"]:.6f}')
    print('-' * 60)

    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch + 1}. Best validation loss: {best_val_loss:.4f}")
        break

torch.save(temp, 'saved_models/10B_Depth_best_model.pth')
print("Training completed.")

Starting training...
Epoch [1/3]
Training Loss: 0.7920, Training Accuracy: 57.08%
Validation Loss: 0.7296, Validation Accuracy: 53.25%
Current LR: 0.001000
------------------------------------------------------------
Epoch [2/3]
Training Loss: 0.6334, Training Accuracy: 64.58%
Validation Loss: 0.6156, Validation Accuracy: 65.75%
Current LR: 0.001000
------------------------------------------------------------
Epoch [3/3]
Training Loss: 0.6061, Training Accuracy: 66.92%
Validation Loss: 0.5622, Validation Accuracy: 68.25%
Current LR: 0.001000
------------------------------------------------------------
Training completed.


In [24]:
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

model.load_state_dict(torch.load('saved_models/DEPTH/10B_Depth_best_model.pth', weights_only=True))

model = model.to(device)

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test = datasets.ImageFolder(root='Real_Labelled_data_d', transform=test_transform)

test_loader = DataLoader(test, batch_size=16, shuffle=False)

test_accuracy, test_loss = evaluate_model(model, test_loader)
print(f'\nFinal Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')



Final Test Loss: 0.5621, Test Accuracy: 72.21%
